<h1>face recognition - For unlock ur apps and folders</h1>
 <p>We will train our model by our faces using open cv .And then whem there is a highly confidence level of 75%<br>
        our application will unlock. We will use our web cam for performing the activity</p>

<h2> step 1 - creat the Training data

In [4]:
import cv2
import numpy as np
# import time


# Load HAAr face classifier 
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load function
# ############################
# creat trainig data
# ############################

def face_extractor(img):
    # this function dectes the face and returns the cropped image
    #If no face is found it will return the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces == ():
        return None
    #crop all faces found
    for(x,y,w,h) in faces:
        cropped_face = img[y:y+h,x:x+w]
        
    return cropped_face

# end of the block 
# ########################

# Intializing the webcam
cap = cv2.VideoCapture(0)
count = 0

# count the 100 samples of your data from the input web cam
while True:
    
#     print(count)
#     time.sleep(0.5)
    ret,frame = cap.read()
    if face_extractor is not None:
        count += 1
        try:
            face = cv2.resize(face_extractor(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        except:
            pass
        #save the photos in a specific folder so that it can be used forther in training
        file_name_path = './faces/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        
        #put count on the images and take live count
        cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,225,0),2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print('Face Not Found')
        pass
    
    if cv2.waitKey(1) == 13 or count == 100: #13 is for enter key
        break
        
cap.release()
cv2.destroyAllWindows()
print('collection of sample completed')    

<ipython-input-4-b28ff3efb556>:21: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


collection of sample completed


# step 2 - train ur model
<a href="https://docs.opencv.org/3.4/df/d25/classcv_1_1face_1_1LBPHFaceRecognizer.html">we use lbhfacerecognizer</a>

In [3]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join
from sklearn.model_selection import train_test_split

# now we will get the training data 
data_path='./faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]

# created arrey for training the data and labels
Training_Data,Labels = [],[]

# open training images 
# creating a numpy arrey to train the data
for i,files in enumerate(onlyfiles):
    image_path = data_path+onlyfiles[i]
    images=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images,dtype=np.uint8))
    Labels.append(i)
#     print(i)
    
    
# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
model = cv2.face.LBPHFaceRecognizer_create()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()


# lets train the model
model.train(np.asarray(Training_Data),np.asarray(Labels))
print('Model is trained Sucessfully')

Model is trained Sucessfully


In [5]:
model.getHistograms()

[array([[0.01041667, 0.        , 0.        , ..., 0.03125   , 0.00694444,
         0.09201389]], dtype=float32),
 array([[0.01215278, 0.00173611, 0.        , ..., 0.01736111, 0.        ,
         0.06423611]], dtype=float32),
 array([[0.01909722, 0.00520833, 0.        , ..., 0.01736111, 0.01388889,
         0.09895834]], dtype=float32),
 array([[0.01215278, 0.00173611, 0.        , ..., 0.01736111, 0.        ,
         0.06423611]], dtype=float32),
 array([[0.01041667, 0.00520833, 0.        , ..., 0.01736111, 0.015625  ,
         0.10590278]], dtype=float32),
 array([[0.01041667, 0.00520833, 0.        , ..., 0.01736111, 0.015625  ,
         0.10590278]], dtype=float32),
 array([[0.01041667, 0.00173611, 0.        , ..., 0.01041667, 0.        ,
         0.09201389]], dtype=float32),
 array([[0.01041667, 0.00173611, 0.        , ..., 0.01041667, 0.        ,
         0.09201389]], dtype=float32),
 array([[0.00868056, 0.        , 0.        , ..., 0.00520833, 0.00694444,
         0.08506945]],

# step3-Test Our Face


In [2]:
import cv2
import numpy as np

# Load HAAr face classifier 
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img,size=0.5):
    
    #Creat image in grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    if faces == ():
        return img,[]
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h,x:x+w]
        roi = cv2.resize(roi,(200,200))
    return img, roi

# open webcam
cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = cap.read()
    
    image,face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)
        
        if confidence >= 90:
            cv2.putText(image, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
        else:
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
    
cap.release()
cv2.destroyAllWindows()
            
            

<ipython-input-2-3b9b7c908517>:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


In [7]:
# recognizer=cv2.face.EigenFaceRecognizer_create()

In [8]:
import cv2

In [9]:
cv2.__version__

'4.4.0'